# Updating and Deleting Data

### Updating Data

In [1]:
UPDATE table
SET column = value, ... ;

ERROR 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'table SET column = value, ...' at line 1


In [2]:
UPDATE birdwatchers.humans
SET country_id = 'us';

ERROR 1054 (42S22): Unknown column 'country_id' in 'field list'


In [3]:
USE birdwatchers;

### Updating Specific Rows

In [4]:
SELECT human_id, name_first, name_last
FROM humans
WHERE name_first = 'Rusty'
AND name_last = 'Osborne';

In [5]:
UPDATE humans
SET name_last = 'Johnson'
WHERE human_id = 3;
SELECT human_id, name_first, name_last
FROM humans
WHERE human_id = 3;

In [6]:
UPDATE humans
SET formal_title = 'Ms.'
WHERE human_id IN(24, 32);

In [7]:
SHOW FULL COLUMNS
FROM humans
LIKE 'formal_title';

Field Type Collation Null Key Default Extra Privileges Comment formal_title varchar(25) utf8mb4_general_ci YES NULL select,insert,update,references

In [9]:
UPDATE humans
SET formal_title = 'Ms.'
WHERE formal_title IN('Miss','Mrs.');
Now that all of the members have either a val;

ERROR 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'Now that all of the members have either a val' at line 1


In [10]:
ALTER TABLE humans
CHANGE COLUMN formal_title formal_title ENUM('Mr.','Ms.');

Stage: 2 of 2 'Enabling keys' 0% of stage done

In [11]:
SHOW WARNINGS;

In [12]:
ALTER TABLE humans
CHANGE COLUMN formal_title formal_title ENUM('Mr.','Ms.','Mr','Ms');
UPDATE humans
SET formal_title = SUBSTRING(formal_title, 1, 2);
ALTER TABLE humans
CHANGE COLUMN formal_title formal_title ENUM('Mr','Ms');

Stage: 2 of 2 'Enabling keys' 0% of stage done

### Limiting Updates

In [13]:
CREATE TABLE prize_winners
(winner_id INT AUTO_INCREMENT PRIMARY KEY,
human_id INT,
winner_date DATE,
prize_chosen VARCHAR(255),
prize_sent DATE);

In [14]:
INSERT INTO prize_winners
(human_id)
SELECT human_id
FROM humans;

### Ordering to Make a Difference

In [15]:
UPDATE prize_winners
SET winner_date = CURDATE()
WHERE winner_date IS NULL
ORDER BY RAND()
LIMIT 2;

### Updating Multiple Tables

In [16]:
UPDATE prize_winners, humans
SET winner_date = NULL,
prize_chosen = NULL,
prize_sent = NULL
WHERE country_id = 'uk'
AND prize_winners.human_id = humans.human_id;

ERROR 1054 (42S22): Unknown column 'country_id' in 'where clause'


In [17]:
UPDATE prize_winners, humans
SET winner_date = CURDATE()
WHERE winner_date IS NULL
AND country_id = 'uk'
AND prize_winners.human_id = humans.human_id
ORDER BY RAND()
LIMIT 2;

ERROR 1054 (42S22): Unknown column 'country_id' in 'where clause'


In [18]:
UPDATE prize_winners
SET winner_date = CURDATE()
WHERE winner_date IS NULL
AND human_id IN
(SELECT human_id
FROM humans
WHERE country_id = 'uk'
ORDER BY RAND())
LIMIT 2;

ERROR 1054 (42S22): Unknown column 'country_id' in 'where clause'


### Handling Duplicates

In [19]:
ALTER TABLE humans
ADD COLUMN better_birders_site TINYINT DEFAULT 0;

In [20]:
INSERT INTO humans
(formal_title, name_first, name_last, email_address, better_birders_site)
VALUES('Mr','Barry','Pilson', 'barry@gomail.com', 1),
('Ms','Lexi','Hollar', 'alexandra@mysqlresources.com', 1),
('Mr','Ricky','Adams', 'ricky@gomail.com', 1)
ON DUPLICATE KEY
UPDATE better_birders_site = 2;

In [21]:
INSERT INTO prize_winners
(human_id)
SELECT human_id
FROM humans
WHERE better_birders_site = 1;

In [22]:
ALTER TABLE humans
ADD COLUMN possible_duplicate TINYINT DEFAULT 0;
CREATE TEMPORARY TABLE possible_duplicates
(name_1 varchar(25), name_2 varchar(25));

In [23]:
INSERT INTO possible_duplicates
SELECT name_first, name_last
FROM
(SELECT name_first, name_last, COUNT(*) AS nbr_entries
FROM humans
GROUP BY name_first, name_last) AS derived_table
WHERE nbr_entries > 1;

In [24]:
UPDATE humans, possible_duplicates
SET possible_duplicate = 1
WHERE name_first = name_1
AND name_last = name_2;

### Deleting Data

In [26]:
DELETE
[WHERE
[ORDER
[LIMIT
FROM table
condition]
BY column]
row_count];

ERROR 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near '[WHERE [ORDER [LIMIT FROM table condition] BY column] row_count]' at line 1


In [27]:
DELETE FROM humans
WHERE name_first = 'Elena'
AND name_last = 'Bokova'
AND email_address LIKE '%yahoo.com';

### Deleting in Multiple Tables

In [28]:
DELETE FROM table[, table]
USING table[, . . . ]
[WHERE condition];

ERROR 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'table[, table] USING table[, . . . ] [WHERE condition]' at line 1


In [29]:
DELETE FROM humans, prize_winners
USING humans JOIN prize_winners
WHERE name_first = 'Elena'
AND name_last = 'Bokova'
AND email_address LIKE '%yahoo.com'
AND humans.human_id = prize_winners.human_id;

### Summary

The UPDATE and DELETE statements are very useful for changing data in tables; they are
essential to managing a MySQL or MariaDB database. They have many possibilities for
effecting changes to tables with ease. You can construct very complex SQL statements
with them to change precisely the data you want to change or to delete exactly the rows
you want to delete. However, it can be confusing and difficult at times. So be careful and
learn these SQL statements well.
If you’re nervous at times about using the UPDATE and DELETE statements, it’s because
you should be. You can change all of the rows in a table with one UPDATE statement, and
you can delete all of the rows in a table with one DELETE statement. On a huge database,
that could be thousands of rows of data changed or deleted in seconds. This is why good
backups are always necessary. Whenever using these two SQL statements, take your
time to be sure you’re right before you execute them. While you’re still learning espe‐
cially, it can be a good idea to make a duplicate of a table with its data using the CREATE
TABLE...SELECT statement before updating or deleting data. This SQL statement was
covered in “Essential Changes” on page 61. This way if you make a major mistake, you
can put the data back as it was before you started.
Because of the problems you can cause yourself and others who will use the databases
on which you will work, practice using the UPDATE and DELETE statements. More than
any other chapter in this book so far, you should make sure to complete the exercises
in the next section.